In [1]:
!pip install lightning datasets torchmetrics --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.7 MB/s eta 0:00:00


In [2]:
import os
import sys

import numpy as np
import pandas as pd
from tqdm.auto import tqdm

from sklearn.preprocessing import MinMaxScaler, StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW, SGD
from torch.optim.lr_scheduler import CosineAnnealingLR
import torch.utils as utils
from torch.utils.data import Dataset, TensorDataset, DataLoader, random_split

import lightning as L
from lightning import seed_everything
from lightning.pytorch.callbacks import TQDMProgressBar

import torchmetrics
from torchmetrics.functional import r2_score, mean_absolute_error, mean_squared_error

import transformers
from transformers import get_linear_schedule_with_warmup

import matplotlib as  mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import google.colab
google.colab.drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/projects/temporature-hackathon/')

Mounted at /content/drive


In [60]:
class CNN_LSTM(L.LightningModule):
    def __init__(self, hidden_size=64, num_layers=10, output_len=358):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_len = output_len
        self.encoder = nn.LSTM(64, hidden_size, num_layers, batch_first=True)
        self.decoder = nn.LSTM(1, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 1),
        )
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(3, 9), padding=(1, 0)),
            nn.GELU()
        )
        self.cnn2 = nn.Sequential(
            nn.Conv1d(16, 64, kernel_size=21, padding=10),
            nn.GELU()
        )

        self.criterion = nn.MSELoss()

        self.training_stats = []

    def forward(self, prev_time_series):
        batch_size = len(prev_time_series)
        prev_time_series = prev_time_series.view(batch_size, 1, -1, 9)
        x = self.cnn1(prev_time_series)
        x = x.squeeze()
        x = self.cnn2(x)
        x = x.view(batch_size, -1, 64)
        encoder_output, (hidden, cell) = self.encoder(x,
         (torch.zeros([self.num_layers, batch_size, self.hidden_size]).type_as(x),
          torch.zeros([self.num_layers, batch_size, self.hidden_size]).type_as(x)
          )
        )
        decoder_input = torch.ones([batch_size, self.output_len, 1]).type_as(x)
        decoder_output, (hidden, cell) = self.decoder(decoder_input, (hidden, cell))
        x = self.fc(decoder_output)
        x = x.squeeze()
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        self.log('train_loss', loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y = y.squeeze()
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_r2', r2_score(y_hat, y), prog_bar=True)
        self.log('val_mae', mean_absolute_error(y_hat, y), prog_bar=True)
        self.log('val_mse', mean_squared_error(y_hat, y), prog_bar=True)
        self.training_stats.append(
            {
                'val_loss': loss,
                'val_r2': r2_score(y_hat, y),
                'val_mae': mean_absolute_error(y_hat, y),
                'val_mse': mean_squared_error(y_hat, y),
            }
        )

    def on_test_epoch_end(self):
        pass

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=0.002)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=200, num_training_steps=1500)
        #scheduler = CosineAnnealingLR(optimizer, T_max=1000)
        scheduler = {
            'scheduler': scheduler,
            'interval': 'step',
            'frequency': 1
        }
        return [optimizer], [scheduler]

In [106]:
class CNN_Transformer(L.LightningModule):
    def __init__(self, hidden_size=64, num_layers=10, output_len=358):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_len = output_len
        self.transformer = nn.Transformer(
            d_model=hidden_size,
            dim_feedforward=hidden_size * 2,
            activation=F.gelu,
            num_encoder_layers=3,
            num_decoder_layers=1,
        )
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, 1),
        )
        self.cnn1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=(3, 9), padding=(1, 0)),
            nn.GELU()
        )
        self.cnn2 = nn.Sequential(
            nn.Conv1d(16, 64, kernel_size=21, padding=10),
            nn.GELU()
        )

        self.criterion = nn.MSELoss()

        self.training_stats = []

    def forward(self, prev_time_series):
        batch_size = len(prev_time_series)
        prev_time_series = prev_time_series.view(batch_size, 1, -1, 9)
        x = self.cnn1(prev_time_series)
        x = x.squeeze()
        x = self.cnn2(x)
        x = x.view(batch_size, -1, 64)
        x = self.transformer(x, x)
        x = self.fc(x[:, :self.output_len, :])
        x = x.squeeze()
        return x

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)

        self.log('train_loss', loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y = y.squeeze()
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log('val_loss', loss, prog_bar=True)
        self.log('val_r2', r2_score(y_hat, y), prog_bar=True)
        self.log('val_mae', mean_absolute_error(y_hat, y), prog_bar=True)
        self.log('val_mse', mean_squared_error(y_hat, y), prog_bar=True)
        self.training_stats.append(
            {
                'val_loss': loss,
                'val_r2': r2_score(y_hat, y),
                'val_mae': mean_absolute_error(y_hat, y),
                'val_mse': mean_squared_error(y_hat, y),
            }
        )

    def on_test_epoch_end(self):
        pass

    def configure_optimizers(self):
        optimizer = AdamW(self.parameters(), lr=0.002)
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=30, num_training_steps=300)
        #scheduler = CosineAnnealingLR(optimizer, T_max=1000)
        scheduler = {
            'scheduler': scheduler,
            'interval': 'step',
            'frequency': 1
        }
        return [optimizer], [scheduler]

In [5]:
output = pd.read_pickle('output.pkl')
train_df = pd.read_pickle('train_df.pkl')
output = torch.tensor(output.values)
train_data = torch.tensor(train_df.values)
output.shape, train_data.shape

(torch.Size([358]), torch.Size([22653, 10]))

In [61]:
model = CNN_LSTM()

In [9]:
input_tensor = torch.load('input.pt')
output_tensor = torch.load('output.pt')

In [10]:
dataset = TensorDataset(input_tensor.type(torch.float32)[:-1], output_tensor.type(torch.float32)[:-1])
len(dataset)

60

In [11]:
train_set, val_set = torch.utils.data.random_split(dataset, [56, 4])

In [12]:
train_dataloader = DataLoader(dataset, batch_size=32, shuffle=False)
val_dataloader = DataLoader(dataset, batch_size=6, shuffle=False)

In [62]:
trainer = L.Trainer(
    accelerator='auto',
    max_epochs=1500,
    callbacks=[TQDMProgressBar(refresh_rate=20)],
)

trainer.fit(model, train_dataloaders=train_dataloader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name      | Type       | Params
-----------------------------------------
0 | encoder   | LSTM       | 332 K 
1 | decoder   | LSTM       | 316 K 
2 | fc        | Sequential | 65    
3 | cnn1      | Sequential | 448   
4 | cnn2      | Sequential | 21.6 K
5 | criterion | MSELoss    | 0     
-----------------------------------------
671 K     Trainable params

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=1500` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=1500` reached.


In [47]:
trainer.test(model, dataloaders=val_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │     109.8829574584961     │
│          val_mae          │     9.113381385803223     │
│          val_mse          │     109.8829574584961     │
│          val_r2           │    -17.73712921142578     │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 109.8829574584961,
  'val_r2': -17.73712921142578,
  'val_mae': 9.113381385803223,
  'val_mse': 109.8829574584961}]

In [107]:
ct_model = CNN_Transformer()

In [108]:
ct_trainer = L.Trainer(
    accelerator='auto',
    max_epochs=300,
    callbacks=[TQDMProgressBar(refresh_rate=20)],
)

ct_trainer.fit(ct_model, train_dataloaders=train_dataloader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: IPU available: False, using: 0 IPUs
INFO:lightning.pytorch.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name        | Type        | Params
--------------------------------------------
0 | transformer | Transformer | 150 K 
1 | fc          | Sequential  | 65    
2 | cnn1        | Sequential  | 448   
3 | cnn2        | Sequential  | 21.6 K
4 | criterion   | MSELoss     | 0     
--------------------------------------------
172 K     Trainable params
0         N

Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [109]:
ct_trainer.test(ct_model, dataloaders=val_dataloader)

INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.10/dist-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=1` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val_loss          │     94.90328216552734     │
│          val_mae          │     8.252747535705566     │
│          val_mse          │     94.90328216552734     │
│          val_r2           │    -14.623472213745117    │
└───────────────────────────┴───────────────────────────┘

[{'val_loss': 94.90328216552734,
  'val_r2': -14.623472213745117,
  'val_mae': 8.252747535705566,
  'val_mse': 94.90328216552734}]

In [96]:
train_df = pd.read_pickle('train_df.pkl')
train_df.head()

,max_temp,min_temp,temp_diff,rainfall,avg_humidity,avg_wind_speed,sunshine,sunshine_hours,sunshine_ratio,avg_temp
date,,,,,,,,,,
1960-01-01,-1.388046,-1.290601,-0.419427,-0.712664,0.212881,-0.718610,-0.021898,-0.878121,-0.712664,-1.6
1960-01-02,-1.481384,-1.328416,-0.625823,-0.712664,1.546328,-1.140763,-1.000000,-0.878121,-0.712664,-1.9
1960-01-03,-0.781352,-0.997541,0.750148,-0.712664,1.106428,0.653387,-1.000000,-0.878121,-0.712664,4.0
1960-01-04,-0.585344,-0.685573,0.337357,-0.712664,0.996453,2.130922,-0.620438,-0.878121,-0.712664,7.5
1960-01-05,-1.472050,-1.574209,0.302958,-0.712664,-1.457365,2.869689,0.197080,-0.878121,-0.712664,-4.6


In [97]:
predict_tensor = (torch.tensor(train_df.iloc[-358 * 3:].drop(columns=['avg_temp'])
                                                       .values).type(torch.float32).unsqueeze(0))
predict_tensor.shape

torch.Size([1, 1074, 9])

In [110]:
model.eval()
ct_model.eval()
output = ct_model(predict_tensor.unsqueeze(0)).squeeze()

In [111]:
output = output.detach().numpy()

In [112]:
output.shape

(358,)

In [113]:
test_df = pd.read_csv('sample_submission.csv')

In [114]:
test_df.head()

,일시,평균기온
0,2023-01-01,10.890710
1,2023-01-02,10.571953
2,2023-01-03,10.486100
3,2023-01-04,10.453379
4,2023-01-05,10.418772


In [115]:
test_df['평균기온'] = output
test_df.head()

,일시,평균기온
0,2023-01-01,8.430505
1,2023-01-02,11.024735
2,2023-01-03,8.183732
3,2023-01-04,7.983667
4,2023-01-05,14.544838


In [116]:
test_df.to_csv('sample_submission.csv', index=False)